In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer
import random

In [2]:
# Definir uma semente aleatória para reprodutibilidade
np.random.seed(42)
random.seed(42)

In [3]:
# Gerar as variáveis simuladas com correlação
idade = np.random.randint(18, 71, 10000)

In [4]:
# Gerar variáveis correlacionadas usando a função multivariada normal
mean_values = [5000, 2000, 0.5, 5]  # Médias das variáveis
correlation_matrix = np.array([
    [1, 0.3, 0.2, -0.1],
    [0.3, 1, -0.1, 0.2],
    [0.2, -0.1, 1, 0.4],
    [-0.1, 0.2, 0.4, 1]
])  # Matriz de correlação

In [5]:
# Gerar dados simulados
simulated_data = np.random.multivariate_normal(mean_values, correlation_matrix, 10000)

renda = simulated_data[:, 0]
divida = simulated_data[:, 1]
utilizacao_credito = np.clip(simulated_data[:, 2], 0, 1)  # Limita a utilização de crédito entre 0 e 1
consultas_recentes = np.maximum(simulated_data[:, 3], 0)  # Garante que o número de consultas recentes seja não negativo


In [6]:
# Gerar função linear das variáveis explicativas
preditor_linear = -7 - 0.01 * idade - 0.0002 * renda + 0.003 * divida - 3 * utilizacao_credito + 0.5 * consultas_recentes


In [7]:
# Calcular probabilidade de default (PD) usando a função de link logit
prob_default = 1 / (1 + np.exp(-preditor_linear))


In [8]:
# Gerar inadimplência como variável Bernoulli com base na probabilidade de default
inadimplencia = np.random.binomial(1, prob_default, 10000)


In [9]:
# Criar dataframe
dados = pd.DataFrame({
    'idade': idade,
    'renda': renda,
    'divida': divida,
    'utilizacao_credito': utilizacao_credito,
    'consultas_recentes': consultas_recentes,
    'inadimplencia': inadimplencia
})

print(dados.head())

   idade        renda       divida  utilizacao_credito  consultas_recentes  \
0     56  5001.087921  1999.455126             0.95862            4.875699   
1     69  5000.719625  2000.810295             1.00000            6.445579   
2     46  4999.716997  2001.196934             0.00000            5.101072   
3     32  5001.297920  2000.322691             0.69576            5.748065   
4     60  4999.152684  2000.239878             1.00000            5.304662   

   inadimplencia  
0              0  
1              0  
2              1  
3              0  
4              0  


In [10]:
# Categorizar a idade
kbin = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
dados['idade_cat'] = kbin.fit_transform(dados[['idade']])

def descritiva2(var1, var2, df):
    cross_tab = pd.crosstab(df[var1], df[var2], normalize='index')
    print(cross_tab)

descritiva2('idade_cat', 'inadimplencia', dados)

print(dados.head())

dados.to_parquet('exercicio.parquet')

#####################################################################################
# Agora é a sua vez: Ajuste uma árvore de decisão, e explore os recursos que fizemos 
# ao longo da aula nesta base de dados ;)

inadimplencia         0         1
idade_cat                        
0.0            0.703339  0.296661
1.0            0.725639  0.274361
2.0            0.752874  0.247126
3.0            0.761631  0.238369
4.0            0.773301  0.226699
   idade        renda       divida  utilizacao_credito  consultas_recentes  \
0     56  5001.087921  1999.455126             0.95862            4.875699   
1     69  5000.719625  2000.810295             1.00000            6.445579   
2     46  4999.716997  2001.196934             0.00000            5.101072   
3     32  5001.297920  2000.322691             0.69576            5.748065   
4     60  4999.152684  2000.239878             1.00000            5.304662   

   inadimplencia  idade_cat  
0              0        3.0  
1              0        4.0  
2              1        2.0  
3              0        1.0  
4              0        4.0  
